# Lezione 12 - Fit di istogrammi

## Introduzione
Il metodo dei minimi quadrati può anche essere applicato per eseguire il fit di istogrammi, in cui un modello di pdf $ f(x, \theta) $ viene adattato ai dati binnati. Il contenuto di ciascun bin è un numero di eventi $ n_{i} $; se il numero di bin e il numero di eventi per bin $ n_{i} $ sono grandi, allora $ n_{i} $ segue la distribuzione di Poisson, essendo un conteggio:

$$ P(n; \mu) = \frac{e^{-\mu} \mu^{n}}{n!} $$

## Fit di istogrammi usando il metodo dei minimi quadrati
Si usa solitamente il $ Q^2 $ di Neyman:

$$ Q^2(\theta) = \sum_{i = 1}^{N}{\left(\frac{n_{i} - f(x_{i}, \theta)}{\sqrt{n_{i}}}\right)^2} $$

L'incertezza $ \sqrt{n_{i}} $ deriva dal fatto che i conteggi seguono la statistica di Poisson

## Fit di istogrammi usando il metodo di maximum likelihood
Nel caso della maximum likelihood, si assume che in ogni bin i conteggi seguono una distribuzione di probabilità poissoniana con valore aspettato $ \mu = F_{i}(\theta) $, che è il numero aspettato di eventi per ciascun bin:

$$ \mathcal{L}(\theta) = \prod_{i = 1}^{N}{\frac{e^{-f(x_{i}, \theta)} F_{i}(\theta)^{n_{i}}}{n_{i}!}} $$

## Scelta del metodo più adatto
- Nel caso di pochi conteggi presenti nei bin, i bin con zero conteggi non contribuiscono al fit nel caso dei minimi quadrati, quindi vi è una perdita di informazione; pertanto è preferibile usare il metodo di maximum likelihood
- Nel caso di molti conteggi presenti nei bin, i due metodi sono equivalenti e spesso il metodo dei minimi quadrati è computazionalmente superiore

## Esempio: segnale gaussiano con rumore esponenziale
Si considera il seguente modello

\begin{align*}
    f(x) &= N_{B} \cdot \exp{(x, \theta_0)} + N_{S} \cdot \mathcal{G}(x; \theta_{1}, \theta_{2}) = \\ 
    &= N_{B} \cdot \frac{1}{\theta_{0}} e^{-\frac{x}{\theta_{0}}} + N_{S} \cdot \frac{1}{\sqrt{2 \pi} \theta_{2}} e^{-\frac{1}{2}\left(\frac{x - \theta_{1}}{\theta_{2}}\right)^2}
\end{align*}

Nel modello sia $ N_{B} $ che $ N_{S} $ sono ignoti, quindi bisogna ricorrere all'extended likelihood

$$ \mathcal{L}_{ext}(x, \theta, N, \nu) = P(N, \nu) \cdot \prod_{i = 1}^{N}{f_{x}(x_{i}, \theta)} $$

### Determinare i parametri del fit
Per determinare i parametri del fit, si deve determinare l'estremo di una funzione con più parametri e in presenza di fluttuazioni statistiche. Per farlo si utilizza la libreria iminuit.

### Definizione del modello per il fit
- La minimizzazione effettuata dal fit si basa sul confronto tra il numero di eventi osservati in ciascuno dei bin e il valore aspettato del numero di eventi in ciascun bin
- Il valore di aspettazione viene calcolato integrando la PDF modello
- Dunque, il modello che verrà dato in input per il fit si basa sulle CDF, non sulle PDF
- Inoltre si deve dare un `bin_edges` come input del modello

```Python
from scipy.stats import norm, expon

def mod_total(bin_edges, N_signal, mu, sigma, N_background, t_0):
    return N_signal * norm.cdf(bin_edges, mu, sigma) + N_background * expon.cdf(bin_edges, 0, t_0)
```

### Impostazione del fit
La cost function viene definita da

```Python
from iminuit.cost import ExtendedBinnedNLL

my_cost_func = ExtendedBinnedNLL(bin_content, bin_edges, mod_total)
```

L'algoritmo di ricerca dell'estremo si inizializza come segue

```Python
my_minuit = Minuit(my_cost_func, N_signal = N_events, mu = sample_mean, sigma = sample_sigma, N_background = N_events, t_0 = 1.0)
my_minuit.migrad()
```

**NOTA BENE**: Spesso conviene fare diversi fit su intervalli diversi, perché alcune regioni sono rilevanti per alcuni parametri ma non per altri. Per esempio

<div align="center">
<img src="Pictures/modello_regioni.png" width=600 height=300 alt="loglikelihood">
</div>

### Determinazione dei parametri
Un primo fit parziale tra $ x = 5 $ e $ x = 15 $ con solo i parametri del rumore esponenziale liberi di fluttuare permette di calcolare una stima preliminare di $ N_{B} $ e $ t_0 $

```Python
# setting the signal to zero for a first background-only preliminary fit
my_minuit.values["N_signal"] = 0
# fixing the value of the signal parameters to a constant value
# (i.e. they are not considered in the fit)
my_minuit.fixed["N_signal", "mu", "sigma"] = True

# temporary mask avoiding the signal region
bin_centres = 0.5 * (bin_edges[1:] + bin_edges[:-1])
my_cost_func.mask = (bin_centres < 5) | (15 < bin_centres)

my_minuit.migrad()
```

Un secondo fit parziale svolto con i valori dei parametri del rumore fissati dal fit precedente permette di stimare $ N_{S} $, $ \mu $ e $ \sigma $

```Python
my_cost_func.mask = None # remove mask
my_minuit.fixed = False # release all parameters
my_minuit.fixed["N_background", "tau"] = True # fix background parameters
my_minuit.values["N_signal"] = N_events - my_minuit.values["N_background"] # do not start at the limit
my_minuit.migrad()
```

Nel fit finale tutti i parametri sono lasciati liberi di variare, l'oggetto minuit prenderà come valori iniziali dei parametri quelli determinati dai due fit precedenti

```Python
my_minuit.fixed = False # release all parameters
my_minuit.migrad()
print(my_minuit.valid)
```

Per l'incertezza sui parametri, nel caso di fit con maximum likelihood, bisogna chiamare la funzione

```Python
my_minuit.minos()
```

## Fit con unbinned maximum likelihood
Nel caso di fit con unbinned maximum likelihood, la funzione modello deve essere una PDF

```Python
def mod_signal_unbinned(x, mu, sigma):
    return norm.pdf(x, mu, sigma)
```

Si usa la seguente cost function

```Python
from iminuit.cost import UnbinnedNLL
my_cost_func_unbinned = UnbinnedNLL(subsample, mod_signal_unbinned)
```

## Fit con minimi quadrati
Quando si applica il metodo dei minimi quadrati, il modello di input riscala la PDF per il numero totale di eventi. Nel caso gaussiano, si ha

```Python
def func_approx(x, N_events, mean, sigma, bin_width):
    return N_events * norm.pdf(x, mean, sigma) * bin_width
```

Si usa la seguente cost function
from iminuit.cost import LeastSquares
least_squares = LeastSquares(bin_centres, bin_content, sigma_y, func_approx)